<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/Decisiontrees_on_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import make_scorer, confusion_matrix, classification_report, precision_score, r2_score, f1_score, recall_score

import time

In [26]:
# import yellowbrick
# from yellowbrick.datasets import load_occupancy
# from yellowbrick.model_selection import FeatureImportances

### Mount data from Google Drive

In [13]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Change dir and check contents

In [14]:
cd '/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data'

/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data


In [15]:
ls

matches_agg.csv  rounds_agg.csv


### 1) load data


In [16]:
%%time
matches_agg = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/matches_agg.csv')
rounds_agg = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/rounds_agg.csv')

CPU times: user 708 ms, sys: 132 ms, total: 840 ms
Wall time: 1.57 s


In [17]:
matches_agg

,file,map,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class
0,esea_match_13770997.dem,de_overpass,25,84.747674,20009.375000,9,16,Team 2,-1
1,esea_match_13779704.dem,de_cache,39,97.655995,21298.717949,18,21,Team 2,-1
2,esea_match_13779770.dem,de_mirage,29,94.112689,18600.000000,14,15,Team 2,-1
3,esea_match_13779775.dem,de_inferno,30,94.452687,19579.166667,16,14,Team 1,1
4,esea_match_13779780.dem,de_inferno,29,102.916448,18168.103448,15,14,Team 1,1
...,...,...,...,...,...,...,...,...,...
13456,esea_match_13829155.dem,de_cache,29,81.956562,18927.586207,16,13,Team 1,1
13457,esea_match_13829165.dem,de_dust2,24,74.316426,18516.304348,15,9,Team 1,1
13458,esea_match_13829169.dem,de_mirage,18,87.696972,16775.000000,15,3,Team 1,1
13459,esea_match_13829171.dem,de_train,24,75.069213,18711.458333,16,8,Team 1,1


In [18]:
rounds_agg

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,winner_team,winner_side,winner_class,winner_side_class,past_halftime
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,Team 2,Terrorist,-1,-1,0
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,Team 2,Terrorist,-1,-1,0
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,Team 2,Terrorist,-1,-1,0
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,Team 2,Terrorist,-1,-1,0
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,Team 1,CounterTerrorist,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,Team 1,Terrorist,1,-1,1
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,Team 1,Terrorist,1,-1,1
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,Team 1,Terrorist,1,-1,1
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,Team 1,Terrorist,1,-1,1


### apply standard scaler

In [19]:
stdscl = StandardScaler()
stdscl

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
std_matches_agg = pd.DataFrame(data=StandardScaler().fit_transform(matches_agg[['num_rounds',	'avg_round_duration',	'avg_equip_val',	'team_1_score',	'team_2_score']]),columns=['num_rounds',	'avg_round_duration',	'avg_equip_val',	'team_1_score',	'team_2_score'])
std_matches_agg[['map','winner_class']] = matches_agg[['map','winner_class']]
std_matches_agg

,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,map,winner_class
0,-0.114617,-0.323955,1.319312,-0.943692,0.768251,de_overpass,-1
1,3.102073,1.318159,2.518032,1.218854,1.888616,de_cache,-1
2,0.804437,0.867402,0.008996,0.257723,0.544178,de_mirage,-1
3,1.034201,0.910655,0.919341,0.738288,0.320104,de_inferno,1
4,0.804437,1.987361,-0.392544,0.498005,0.320104,de_inferno,1
...,...,...,...,...,...,...,...
13456,0.804437,-0.679023,0.313558,0.738288,0.096031,de_cache,1
13457,-0.344381,-1.650952,-0.068817,0.498005,-0.800261,de_dust2,1
13458,-1.722962,0.051236,-1.687732,0.498005,-2.144700,de_mirage,1
13459,-0.344381,-1.555188,0.112620,0.738288,-1.024334,de_train,1


In [21]:
cols_to_standardize=['round',	'round_duration',	'ct_eq_val', 't_eq_val', 'final_ct_alive', 'final_t_alive'] #start and end seconds not used
cols_to_copy=['map','is_bomb_planted','bomb_site','winner_class','winner_side_class','past_halftime']

std_rounds_agg = pd.DataFrame(data=StandardScaler().fit_transform(rounds_agg[cols_to_standardize]),columns=cols_to_standardize)
std_rounds_agg[cols_to_copy] = rounds_agg[cols_to_copy]
std_rounds_agg

,round,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,map,is_bomb_planted,bomb_site,winner_class,winner_side_class,past_halftime
0,-1.584582,-0.734664,-1.399625,-1.609951,-0.916157,-0.242764,de_overpass,1,-1,-1,-1,0
1,-1.459545,1.091897,-1.209110,0.119316,-0.290183,0.392589,de_overpass,0,0,-1,-1,0
2,-1.334508,-0.912514,-1.080513,0.102194,-0.916157,1.663297,de_overpass,0,0,-1,-1,0
3,-1.209471,0.244623,0.562679,0.575888,-0.916157,-0.242764,de_overpass,1,1,-1,-1,0
4,-1.084434,-1.363770,-1.294842,0.250580,0.961767,-0.878118,de_overpass,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
340328,0.541048,-1.411909,-1.380574,0.410380,-0.916157,1.027943,de_mirage,0,0,1,-1,1
340329,0.666085,-1.012170,-0.247010,0.884074,-0.916157,1.027943,de_mirage,0,0,1,-1,1
340330,0.791123,-1.314179,-0.370845,0.872660,-0.916157,1.027943,de_mirage,0,0,1,-1,1
340331,0.916160,-1.236340,-0.137464,0.815588,-0.916157,0.392589,de_mirage,0,0,1,-1,1


In [58]:
def hyperparameter_exhasutive_search (model, parameters, scoring, x_train, x_test, y_train, y_test):
  for i in range (len(scoring)):

        startTime = time.time()

        clf = GridSearchCV(estimator=model, param_grid=parameters, scoring=scoring[i])
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        endTime = time.time()

        clf_report = classification_report(y_test,y_pred)

        print('Optimizing hyperparameters for ', scoring[i])
        print('\nElapsed Time (s):', round((endTime -startTime),3))
        
        print("\nBest parameters:", clf.best_params_)

        print('\nClassification report:\n', clf_report)

        print('-----------------------------------------------------------------------------\n')

  print('search complete!')

In [30]:
matches_agg.columns

Index(['file', 'map', 'num_rounds', 'avg_round_duration', 'avg_equip_val',
       'team_1_score', 'team_2_score', 'winner', 'winner_class'],
      dtype='object')

In [47]:
matches_agg[matches_agg.isna().any(axis=1)]

,file,map,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class
1234,esea_match_13781946.dem,de_mirage,20,NaN,17931.25,4,16,Team 2,-1


In [52]:
matches_agg.dropna(inplace=True)
matches_agg[matches_agg.isna().any(axis=1)]

,file,map,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class


In [83]:
test_size=0.3
columns=['num_rounds','avg_round_duration', 'avg_equip_val'] #team1score and team2score would make it easy

x_train, x_test, y_train, y_test = train_test_split(matches_agg[columns], matches_agg['winner_class'], test_size=test_size, random_state=0)

In [84]:
y_test

4146    -1
6788    -1
10658    1
10714    1
5137     1
        ..
4902    -1
7835    -1
11365    1
11887    1
11947    1
Name: winner_class, Length: 4038, dtype: int64

In [85]:
model = DecisionTreeClassifier()
parameters = {'splitter':('best', 'random'), 'max_depth':[1,2,3,4,5,6],'criterion':('gini', 'entropy'),}

#the average function is binary by default. This causes error in the GridSearchCV function for Decision Trees. therefore pass a score function with custom average settings
average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
scoring =[
    make_scorer(precision_score,average=average_method), 
    make_scorer(recall_score,average=average_method),
    make_scorer(f1_score,average=average_method)
    ]

hyperparameter_exhasutive_search(model=model, parameters=parameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

Optimizing hyperparameters for  make_scorer(precision_score, average=micro)

Elapsed Time (s): 1.526

Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.48      0.24      0.32      1858
           0       0.23      0.07      0.10       163
           1       0.54      0.82      0.65      2017

    accuracy                           0.52      4038
   macro avg       0.42      0.38      0.36      4038
weighted avg       0.50      0.52      0.48      4038

-----------------------------------------------------------------------------



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Optimizing hyperparameters for  make_scorer(recall_score, average=micro)

Elapsed Time (s): 1.426

Best parameters: {'criterion': 'entropy', 'max_depth': 4, 'splitter': 'random'}

Classification report:
               precision    recall  f1-score   support

          -1       0.46      0.42      0.44      1858
           0       0.00      0.00      0.00       163
           1       0.54      0.62      0.58      2017

    accuracy                           0.51      4038
   macro avg       0.33      0.35      0.34      4038
weighted avg       0.48      0.51      0.49      4038

-----------------------------------------------------------------------------

Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 1.475

Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.48      0.24      0.32      1858
           0       0.23      

In [72]:
rounds_agg[rounds_agg.isna().any(axis=1)]

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,winner_team,winner_side,winner_class,winner_side_class,past_halftime
31205,esea_match_13781946.dem,de_mirage,1,NaN,NaN,NaN,3650,4350,PISTOL_ROUND,0,3,1,1,Team 2,Terrorist,-1,-1,0
31206,esea_match_13781946.dem,de_mirage,2,NaN,NaN,NaN,6750,18850,ECO,0,4,0,0,Team 2,Terrorist,-1,-1,0
31207,esea_match_13781946.dem,de_mirage,3,NaN,NaN,NaN,1000,22300,ECO,0,5,1,-1,Team 2,Terrorist,-1,-1,0
31208,esea_match_13781946.dem,de_mirage,4,NaN,NaN,NaN,23100,25300,NORMAL,5,0,0,0,Team 1,CounterTerrorist,1,1,0
31209,esea_match_13781946.dem,de_mirage,5,NaN,NaN,NaN,26850,25300,NORMAL,1,2,1,-1,Team 2,Terrorist,-1,-1,0
31210,esea_match_13781946.dem,de_mirage,6,NaN,NaN,NaN,12100,27150,SEMI_ECO,0,4,0,0,Team 2,Terrorist,-1,-1,0
31211,esea_match_13781946.dem,de_mirage,7,NaN,NaN,NaN,16650,24050,NORMAL,0,2,1,1,Team 2,Terrorist,-1,-1,0
31212,esea_match_13781946.dem,de_mirage,8,NaN,NaN,NaN,3000,27850,ECO,1,3,1,1,Team 2,Terrorist,-1,-1,0
31213,esea_match_13781946.dem,de_mirage,9,NaN,NaN,NaN,29100,29800,NORMAL,2,0,1,-1,Team 1,CounterTerrorist,1,1,0
31214,esea_match_13781946.dem,de_mirage,10,NaN,NaN,NaN,25850,26050,NORMAL,0,3,1,-1,Team 2,Terrorist,-1,-1,0


In [73]:
rounds_agg.dropna(inplace=True)
rounds_agg[rounds_agg.isna().any(axis=1)]

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,winner_team,winner_side,winner_class,winner_side_class,past_halftime


In [74]:
rounds_agg.columns

Index(['file', 'map', 'round', 'start_seconds', 'end_seconds',
       'round_duration', 'ct_eq_val', 't_eq_val', 'round_type',
       'final_ct_alive', 'final_t_alive', 'is_bomb_planted', 'bomb_site',
       'winner_team', 'winner_side', 'winner_class', 'winner_side_class',
       'past_halftime'],
      dtype='object')

In [77]:
test_size=0.3
columns=['round', 'start_seconds', 'end_seconds',
       'round_duration', 'ct_eq_val', 't_eq_val','is_bomb_planted', 'bomb_site', #removing ct alive and t alive counts
       'past_halftime']

x_train, x_test, y_train, y_test = train_test_split(rounds_agg[columns], rounds_agg['winner_side_class'], test_size=test_size, random_state=0)

In [79]:
model = DecisionTreeClassifier()
parameters = {'splitter':('best', 'random'), 'max_depth':[1,2,3,4,5,6],'criterion':('gini', 'entropy'),}

hyperparameter_exhasutive_search(model=model, parameters=parameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

Optimizing hyperparameters for  make_scorer(precision_score, average=micro)

Elapsed Time (s): 53.961

Best parameters: {'criterion': 'gini', 'max_depth': 6, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.83      0.80     52301
           1       0.81      0.76      0.78     49793

    accuracy                           0.79    102094
   macro avg       0.79      0.79      0.79    102094
weighted avg       0.79      0.79      0.79    102094

-----------------------------------------------------------------------------

Optimizing hyperparameters for  make_scorer(recall_score, average=micro)

Elapsed Time (s): 54.451

Best parameters: {'criterion': 'gini', 'max_depth': 6, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.83      0.80     52301
           1       0.81      0.76      0.78     49793

    accuracy               

In [81]:
test_size=0.3
columns=['round', 'start_seconds', 'end_seconds',
       'round_duration', 'ct_eq_val', 't_eq_val','is_bomb_planted', 'bomb_site', #removing ct alive and t alive counts
       'past_halftime']

x_train, x_test, y_train, y_test = train_test_split(rounds_agg[columns], rounds_agg['winner_class'], test_size=test_size, random_state=0)

In [82]:
model = DecisionTreeClassifier()
parameters = {'splitter':('best', 'random'), 'max_depth':[1,2,3,4,5,6],'criterion':('gini', 'entropy'),}

hyperparameter_exhasutive_search(model=model, parameters=parameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

Optimizing hyperparameters for  make_scorer(precision_score, average=micro)

Elapsed Time (s): 53.556

Best parameters: {'criterion': 'entropy', 'max_depth': 6, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.79      0.79     49991
           1       0.80      0.78      0.79     52103

    accuracy                           0.79    102094
   macro avg       0.79      0.79      0.79    102094
weighted avg       0.79      0.79      0.79    102094

-----------------------------------------------------------------------------

Optimizing hyperparameters for  make_scorer(recall_score, average=micro)

Elapsed Time (s): 53.387

Best parameters: {'criterion': 'entropy', 'max_depth': 6, 'splitter': 'best'}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.79      0.79     49991
           1       0.80      0.78      0.79     52103

    accuracy         

In [89]:
from sklearn import tree

In [93]:
plottingmodel = DecisionTreeClassifier(criterion='entropy',max_depth=6,splitter='best')
plottingmodel.fit(x_train,y_train)
y_pred = plottingmodel.predict(x_test)

fig, axes = plt.subplots(figsize = (20,20),dpi=600)
plot_tree(plottingmodel, feature_names=columns,  filled=False)

[Text(4850.102459016393, 8412.857142857143, 'num_rounds <= 29.5\nentropy = 1.208\nsamples = 9422\nvalue = [4223, 400, 4799]'),
 Text(2439.344262295082, 7118.571428571428, 'num_rounds <= 27.5\nentropy = 1.025\nsamples = 7972\nvalue = [3638, 28, 4306]'),
 Text(1219.672131147541, 5824.285714285715, 'avg_equip_val <= 18052.099\nentropy = 0.992\nsamples = 6256\nvalue = [2803, 0, 3453]'),
 Text(609.8360655737705, 4530.0, 'avg_round_duration <= 81.604\nentropy = 0.999\nsamples = 2436\nvalue = [1257, 0, 1179]'),
 Text(304.91803278688525, 3235.7142857142862, 'num_rounds <= 22.5\nentropy = 0.979\nsamples = 613\nvalue = [359, 0, 254]'),
 Text(152.45901639344262, 1941.4285714285716, 'avg_equip_val <= 17232.632\nentropy = 0.91\nsamples = 335\nvalue = [226, 0, 109]'),
 Text(76.22950819672131, 647.1428571428569, 'entropy = 0.715\nsamples = 117\nvalue = [94, 0, 23]'),
 Text(228.68852459016392, 647.1428571428569, 'entropy = 0.968\nsamples = 218\nvalue = [132, 0, 86]'),
 Text(457.37704918032784, 1941.42